In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../vacation_search/weatherpy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Rikitea,PF,77.49,broken clouds,-23.1203,-134.9692,People ThankYou
1,3,Abu Dhabi,AE,84.31,clear sky,24.4667,54.3667,Beach Rotana
2,4,Avarua,CK,76.96,overcast clouds,-21.2078,-159.7750,Paradise Inn
3,8,Saint George,US,82.99,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
4,10,Kavieng,PG,83.08,light rain,-2.5744,150.7967,Nusa Island Retreat


In [4]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Maragogi']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Maragogi']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Olinda']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Pitimbu'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Touros'] 
vacation_stop1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
144,551,Olinda,BR,78.53,mist,-8.0089,-34.8553,Flat Quatro Rodas Olinda


In [22]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.iloc[0,5:7].to_numpy()
end = vacation_end.iloc[0,5:7].to_numpy()
stop1 = vacation_stop1.iloc[0,5:7].to_numpy()
stop2 = vacation_stop2.iloc[0,5:7].to_numpy()
stop3 = vacation_stop3.iloc[0,5:7].to_numpy()

start

array([-9.0122, -35.2225], dtype=object)

In [23]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure(center=(38, -122), zoom_level = 7)
ukiah = start
carson_city = stop1
laguna = stop2
half_moon_bay = stop3 
roadtrip = gmaps.directions_layer(ukiah, ukiah, 
                                  waypoints=[carson_city, laguna, half_moon_bay],
                                  travel_mode="DRIVING")

In [24]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, 
                          vacation_stop2, vacation_stop3]
                         ,ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,328,Maragogi,BR,76.95,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
1,551,Olinda,BR,78.53,mist,-8.0089,-34.8553,Flat Quatro Rodas Olinda
2,122,Pitimbu,BR,75.79,broken clouds,-7.4706,-34.8086,Reserva do Abiaí
3,395,Touros,BR,76.60,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON


In [25]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(marker_layer)
fig.add_layer(roadtrip)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))